### Build a Simple LLM Application with LangChain Expression Language (LCEL)
In this quickstart we'll show you how to build a simple LLM application with LangChain. This application will translate text from English into another language. This is a relatively simple LLM application - it's just a single LLM call plus some prompting. Still, this is a great way to get started with LangChain - a lot of features can be built with just some prompting and an LLM call!

After seeing this video, you'll have a high level overview of:

- Using language models

- Using PromptTemplates and OutputParsers

- Using LCEL to chain components together

- Debugging and tracing your application using LangSmith

- Deploying your application with LangServe

### Open AI API Key and Open Source models--Llama3, Gemma2, mistral--Groq

In [2]:
import os
from dotenv import load_dotenv
load_dotenv()

#create OpenAI API key
import openai
openai.api_key=os.getenv("OPENAI_API_KEY")
#create Groq API key
groq_api_key = os.getenv("GROQ_API_KEY")
groq_api_key

'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'

In [10]:
from langchain_groq import ChatGroq
from langchain_openai import ChatOpenAI
model = ChatGroq(model="openai/gpt-oss-120b", groq_api_key=groq_api_key)
model

ChatGroq(profile={'max_input_tokens': 131072, 'max_output_tokens': 32768, 'image_inputs': False, 'audio_inputs': False, 'video_inputs': False, 'image_outputs': False, 'audio_outputs': False, 'video_outputs': False, 'reasoning_output': True, 'tool_calling': True}, client=<groq.resources.chat.completions.Completions object at 0x00000238A01D28C0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x00000238A01D3280>, model_name='openai/gpt-oss-120b', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [12]:
from langchain_core.messages import HumanMessage, SystemMessage
messages = [
    SystemMessage(content="Translate the following from English to French"),
    HumanMessage(content="Hello How are you?")
]

result = model.invoke(messages)
result

AIMessage(content='Bonjour, comment ça va\u202f?', additional_kwargs={'reasoning_content': 'We need to translate "Hello How are you?" to French. Probably "Bonjour, comment ça va ?" or "Bonjour, comment vas-tu ?" The instruction: translate from English to French. Provide translation. Probably just the translation.'}, response_metadata={'token_usage': {'completion_tokens': 63, 'prompt_tokens': 86, 'total_tokens': 149, 'completion_time': 0.132575363, 'completion_tokens_details': {'reasoning_tokens': 47}, 'prompt_time': 0.00320556, 'prompt_tokens_details': None, 'queue_time': 0.004771778, 'total_time': 0.135780923}, 'model_name': 'openai/gpt-oss-120b', 'system_fingerprint': 'fp_c82faf3ca0', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--019b331e-a188-7752-a0c4-9d61686126c4-0', usage_metadata={'input_tokens': 86, 'output_tokens': 63, 'total_tokens': 149, 'output_token_details': {'reasoning': 47}})

### What problem does a string parser solve?

LLMs don’t just return plain text. They usually return a message object (with metadata, roles, tokens, etc.).
A string parser extracts only the text content so you can:

Print it, Store it, Pass it to another function, Feed it into another chain or tool
```bash
from langchain_core.output_parsers import StrOutputParser
output_parser = StrOutputParser()
chain = prompt | llm | output_parser
response = chain.invoke({"input": "What is LangSmith?"})
print(response)
```
1. AIMessage(content="LangSmith is a platform for...")

With a string parser you get:

2. LangSmith is a platform for...


In [13]:
from langchain_core.output_parsers import StrOutputParser
parser = StrOutputParser()
parser.invoke(result)

'Bonjour, comment ça va\u202f?'

#### Using LCEL-chain the components

In [14]:
chain = model|parser
chain.invoke(messages)

'Bonjour, comment ça va\u202f?'

#### Prompt Template

In [15]:
from langchain_core.prompts import ChatPromptTemplate
generic_template = "Translate the following into {language}:"

prompt = ChatPromptTemplate.from_messages(
    [("system", generic_template),
     ("user", "{text}")]
)

| Code               | Meaning                |
| ------------------ | ---------------------- |
| `chain.invoke(x)`  | Run the chain once     |
| `llm.invoke(x)`    | Run the model directly |
| `prompt.invoke(x)` | Format the prompt only |


In [18]:
result = prompt.invoke({"language":"French", "text": "Hello"})

In [19]:
result.to_messages()

[SystemMessage(content='Translate the following into French:', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Hello', additional_kwargs={}, response_metadata={})]

#### Chaining together componenets with LCEL

In [20]:
chain = prompt|model|parser
chain.invoke({"language":"French", "text": "Hello"})

'Bonjour'